In [77]:
# import requests
# import pandas as pd
# url = "https://earthquake.usgs.gov/fdsnws/event/1/query"

# params = {
#     "format": "geojson",
#     "starttime": "2023-01-01",
#     "endtime": "2023-12-31",
#     "minmagnitude": 5
# }
# response = requests.get(url, params=params, timeout=1000)
# data = response.json()
# features = data.get("features", [])
# print(features)



In [78]:
import requests
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss, roc_auc_score

def fetch_usgs_data(start_date, end_date, min_magnitude=2.5):
    url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
    
    try:
        params = {
            "format": "geojson",
            "starttime": start_date,
            "endtime": end_date,
            "minmagnitude": min_magnitude
        }
        
        response = requests.get(url, params=params, timeout=1000)
        response.raise_for_status()
        
        data = response.json()
        features = data.get("features", [])
        
        records = []
        for feature in features:
            props = feature["properties"]
            geometry = feature["geometry"]["coordinates"]
            records.append({
                "time": pd.to_datetime(props["time"], unit="ms"),
                "latitude": geometry[1],
                "longitude": geometry[0],
                "depth": geometry[2],
                "magnitude": props["mag"],
                "event_occurred": 1  
            })
        
        df = pd.DataFrame(records)
        negative_samples = pd.DataFrame({
            "latitude": np.random.uniform(df['latitude'].min(), df['latitude'].max(), 100),
            "longitude": np.random.uniform(df['longitude'].min(), df['longitude'].max(), 100),
            "depth": np.random.uniform(df['depth'].min(), df['depth'].max(), 100),
            "magnitude": np.zeros(100),
            "event_occurred": 0
        })
        
        df = pd.concat([df, negative_samples], ignore_index=True)
        return df
    
    except requests.RequestException as e:
        print(f"Error fetching USGS data: {e}")
        return pd.DataFrame()

In [79]:
def prepare_classification_data(df, lookback=30, test_size=0.2):
    features = ['latitude', 'longitude', 'depth', 'magnitude']
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df[features])
    target = df['event_occurred'].values
    X, y = [], []
    for i in range(len(scaled_data) - lookback):
        X.append(scaled_data[i:i+lookback])
        y.append(target[i+lookback])
    
    X, y = np.array(X), np.array(y)
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42
    )
    return X_train, X_test, y_train, y_test, scaler

In [80]:
def build_probability_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(64, activation='relu', input_shape=input_shape, return_sequences=True),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.LSTM(32, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')  # Probability output
    ])
    model.compile(
        optimizer='adam', 
        loss='binary_crossentropy', 
        metrics=['accuracy', tf.keras.metrics.AUC()]
    )
    
    return model

In [81]:
def evaluate_probability_model(model, X_test, y_test):
    y_pred_proba = model.predict(X_test).flatten()
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    metrics = {
        'Log Loss': log_loss(y_test, y_pred_proba),
        'ROC AUC': roc_auc_score(y_test, y_pred_proba),
        'Accuracy': np.mean(y_pred == y_test),
    }
    return metrics

In [82]:
def predict_earthquake_probability(model, input_data, scaler):
    scaled_input = scaler.transform(input_data)
    scaled_input = scaled_input.reshape(1, -1, 4)
    probability = model.predict(scaled_input)[0][0]
    return probability * 100

In [86]:
import os

def main_earthquake_probability_prediction(
    start_date='2023-12-01', 
    end_date='2024-01-01', 
    min_magnitude=2.5, 
    lookback=30
):
    earthquake_data = fetch_usgs_data(start_date, end_date, min_magnitude)
    if earthquake_data.empty:
        print("No earthquake data retrieved.")
        return None

    X_train, X_test, y_train, y_test, scaler = prepare_classification_data(
        earthquake_data, lookback=lookback
    )
    
    model = build_probability_model(input_shape=(lookback, 4))
    model.fit(
        X_train, y_train, 
        validation_split=0.2, 
        epochs=2, 
        batch_size=32, 
        verbose=1
    )
    
    model_save_path = "../src/server/neural_network/eq_model"
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
    model.save(f"{model_save_path}.keras")
    print(f"Model saved at: {model_save_path}")
    
    metrics = evaluate_probability_model(model, X_test, y_test)
    print("Model Performance Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

    example_input = np.array([[35.0, -117.0, 10.0, 4.0]])
    earthquake_probability = predict_earthquake_probability(model, example_input, scaler)
    
    print(f"\nProbability of Earthquake at (35.0, -117.0): {earthquake_probability:.2f}%")
    
    return {
        'model': model, 
        'metrics': metrics, 
        'scaler': scaler
    }

result = main_earthquake_probability_prediction()


Epoch 1/2


/home/hamza/Dev/analytics_and_modeling/eq_forecaster/notebooks/eq_forecast_env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


62/62 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.9640 - auc_6: 0.6536 - loss: 0.3496 - val_accuracy: 0.9576 - val_auc_6: 1.0000 - val_loss: 0.0627
Epoch 2/2
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.9754 - auc_6: 0.9961 - loss: 0.0377 - val_accuracy: 0.9960 - val_auc_6: 1.0000 - val_loss: 0.0084
Model saved at: ../src/server/neural_network/eq_model
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step
Model Performance Metrics:
Log Loss: 0.01568338938877876
ROC AUC: 0.9998477234658139
Accuracy: 0.9951534733441034


/home/hamza/Dev/analytics_and_modeling/eq_forecaster/notebooks/eq_forecast_env/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 396ms/step

Probability of Earthquake at (35.0, -117.0): 51.77%
